# Roberta Classifier on IMDB: baseline


In [1]:
import json
import os
from typing import List

%pip install datasets
%pip install transformers
import torch
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments


ROOT_DIR = "drive/My Drive/Colab Notebooks/nlp/results/imdb_baseline"
if not os.path.exists(ROOT_DIR):
    os.mkdir(ROOT_DIR)

In [2]:
def get_datasets(dataset_name, train_size, val_size=1_000, test_size=None, random_seed: int = 42):
    """Returns """
    dataset = load_dataset(dataset_name, split="train")
    test_dataset = load_dataset(dataset_name, split="test")
    # We want test and validation data to be the same for every experiment
    if test_size:
        test_dataset = test_dataset.train_test_split(test_size=test_size, seed=random_seed)["test"]
    train_val_split = dataset.train_test_split(test_size=val_size, seed=random_seed)
    # Validation and test sets
    train_dataset = train_val_split["train"].train_test_split(train_size=train_size, seed=random_seed)["train"]
    val_dataset = train_val_split["test"]
    return train_dataset, val_dataset, test_dataset


class DataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        
    def __call__(self, examples: List[dict]):
        labels = [example['label'] for example in examples]
        texts = [example['text'] for example in examples]
        tokenizer_output = self.tokenizer(texts, truncation=True, padding=True)
        return {
            'labels': torch.tensor(labels), 
            'input_ids': torch.tensor(tokenizer_output['input_ids']), 
            'attention_mask': torch.tensor(tokenizer_output['attention_mask'])
            }


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [3]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base', use_fast=True)
data_collator = DataCollator(tokenizer)

In [5]:
TRAIN_SIZES = [20, 100, 1_000, 10_000]
for train_size in TRAIN_SIZES:
    train_dataset, val_dataset, test_dataset = get_datasets("imdb", train_size, val_size=1_000, test_size=5_000)
    print(f"Train size: {len(train_dataset)}, Validation size: {len(val_dataset)}, Test size: {len(test_dataset)}")
    print(train_dataset[0])
    print(val_dataset[0])
    print(test_dataset[0])
    output_dir = os.path.join(ROOT_DIR, f"train_size_{train_size}")

    model = AutoModelForSequenceClassification.from_pretrained('roberta-base', return_dict=True)

    # https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments
    training_args = TrainingArguments(
        learning_rate=3e-5,
        weight_decay=0.01,
        output_dir=output_dir,
        num_train_epochs=6,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=2,  # actual batch size: 16 (as suggested in Bert paper)
        warmup_steps=0,  # don't have any intuition for the right value here
        logging_dir=output_dir,
        logging_steps=5,
        load_best_model_at_end=True,
        evaluation_strategy='epoch',
        remove_unused_columns=False,
        no_cuda=False,
        metric_for_best_model='eval_accuracy'
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        compute_metrics=compute_metrics
        
    )

    trainer.train()

    test_result = trainer.evaluate(test_dataset)

    print(test_result)

    with open(os.path.join(output_dir, 'test_result.json'), 'w') as f:
        json.dump(test_result, f, indent=4)

    print()

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-3dfb39ac59303a4a.arrow and /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-be0d19c5c8e15bf8.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-3bc54fc53ade2e9b.arrow and /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-7c318f8395866af8.arrow
Loading cached s

Train size: 20, Validation size: 1000, Test size: 5000
{'label': 0, 'text': 'After hearing the word of mouth of just how bad this film is I took the plunge and bought the DVD. That said everything previously mentioned about this film is true. For a film that claimed to have a budget in the millions it just does not show on the screen at all. The list of problems with the film could drag on forever. Chief amongst them is the film is simply too long. It dragged on for a few minutes short of 3 hours. Nearly an hour probably could have been cut off the run time had the editor simply removed the overabundance of scenes dealing with nothing more then the main character wandering around aimlessly. <br /><br />Secondly, as many had pointed out from the "trailers", the special effects are anything but special. The tripods looked OK in a few shots here and there but beyond that everything was grade-Z 1970\'s or 1980\'s quality. Probably the worst effects of all were the horses, which stiffly tot

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.695341,0.488000,0.000000,0.000000,0.000000
1,No log,0.699740,0.488000,0.000000,0.000000,0.000000
2,No log,0.701761,0.488000,0.000000,0.000000,0.000000
3,No log,0.702497,0.488000,0.000000,0.000000,0.000000
4,0.944078,0.703700,0.488000,0.000000,0.000000,0.000000
5,0.944078,0.704191,0.488000,0.000000,0.000000,0.000000


{'eval_loss': 0.6939023733139038, 'eval_accuracy': 0.5012, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'epoch': 5.666666666666667, 'total_flos': 39694639169520}



...

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-3dfb39ac59303a4a.arrow and /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-be0d19c5c8e15bf8.arrow
Loading cached split indices for dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-3bc54fc53ade2e9b.arrow and /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3/cache-7c318f8395866af8.arrow


Train size: 10000, Validation size: 1000, Test size: 5000
{'label': 0, 'text': "I went to the movie as a Sneak Preview in Austria. So didn't have an idea what I am going to see. The story is very normal. The movie is very long , I believe it could have cut to 1/2 without causing any problems to the story. Its the type of movie you can see in a boring night which you want to get bored more ! Ashton Kutcher was very good . Kevin Costner is OK. The movie is speaking about the US Coast Guards, how they are trained , their life style and the problems they face. As there aren't much effects in the movie. So if you want to watch it , then no need to waste your money and time going to the Cinema. Would be more effective to watch it at home when it gets on DVDs."}
{'label': 0, 'text': 'This is your typical Priyadarshan movie--a bunch of loony characters out on some silly mission. His signature climax has the entire cast of the film coming together and fighting each other in some crazy moshpit o

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.157812,0.203382,0.932000,0.934489,0.922053,0.947266
2,0.328357,0.240676,0.932000,0.935484,0.909594,0.962891
3,0.063318,0.322237,0.933000,0.933990,0.942346,0.925781
4,0.004620,0.365045,0.935000,0.935580,0.949698,0.921875
5,0.093207,0.374098,0.939000,0.940604,0.937864,0.943359
6,0.081525,0.398375,0.941000,0.942214,0.944990,0.939453


{'eval_loss': 0.3703303635120392, 'eval_accuracy': 0.94, 'eval_f1': 0.9399279134961954, 'eval_precision': 0.9388, 'eval_recall': 0.9410585404971933, 'epoch': 6.0, 'total_flos': 21426663048011040}



Interestingly, model where training loss is close 0 (0.0008 compared to 0.1673) still has the same accuracy (0.940 instead of 0.938) as model without overfitting (even though validation loss is way worse: 0.398 compared to 0.213).

